# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# constants

MODEL_GPT = "gpt-4o-mini"
MODEL_LLAMA = "llama3.2"

In [3]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")

if api_key and api_key.startswith("sk-proj-") and len(api_key) > 10:
    print("API key looks good so far.")
else:
    print("API key is not set or invalid. Please check your .env file.")

openai = OpenAI()

API key looks good so far.


In [4]:
# here is the question; type over this to ask something new

system_prompt = "You are a helpful assistant that explains code snippets in detail. \
    You are also a Python expert and can help with Python programming questions. \
    Response in markdown. Give detailed explanations and examples when needed."

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question},
]


def create_answer(question, ai_model):
    response = openai.chat.completions.create(
        model=ai_model, messages=messages, stream=True
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [6]:
# Get gpt-4o-mini to answer, with streaming
create_answer(question, MODEL_GPT)

Sure! Let's break down the code snippet you provided:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

### Explanation

1. **Context of `yield from`**:
   - The keyword `yield` is used in Python functions to define a generator. A generator is a special type of iterator that allows you to iterate over a sequence of values without storing the entire sequence in memory at once. 
   - The `yield from` expression is used to delegate part of a generator’s operations to another iterable. In this case, it’s delegating to a set comprehension.

2. **Set Comprehension**:
   - The expression `{book.get("author") for book in books if book.get("author")}` is a set comprehension. It's a concise way to create a set based on an iterable. 
   - A set is created that includes the results of `book.get("author")` for each `book` in the `books` iterable, but only if `book.get("author")` returns a truthy value (meaning it's not `None` or empty).

3. **Breaking Down the Set Comprehension**:
   - `for book in books`: This iterates over each item in `books`, where `books` is expected to be a collection (like a list) of book dictionaries.
   - `book.get("author")`: This retrieves the value associated with the key `"author"` in each `book` dictionary. If the key doesn't exist, it returns `None`.
   - `if book.get("author")`: This condition filters out any books that do not have an author. Only the authors that are valid (non-empty and not `None`) will be included in the resulting set.

4. **Resulting Set**:
   - The result of the set comprehension is a set of unique author names from all the books in the `books` collection. Since it's a set, any duplicate author names will be eliminated.

5. **Using `yield from`**:
   - `yield from` will yield each of the unique authors one at a time as part of the generator's output. This allows a caller of the generator to iterate over the authors directly as if they were yielded one by one.

### Example

Here's a practical example to illustrate how this works:

```python
def get_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Sample book list
books = [
    {"title": "Book A", "author": "Author 1"},
    {"title": "Book B", "author": "Author 2"},
    {"title": "Book C"},  # No author
    {"title": "Book D", "author": "Author 1"},  # Duplicate author
    {"title": "Book E", "author": "Author 3"}
]

# Using the generator
for author in get_authors(books):
    print(author)
```

### Output

```
Author 1
Author 2
Author 3
```

### Summary
- The code snippet generates a set of unique author names from a collection of book dictionaries, skipping any that do not have an associated author, and then yields each author one at a time. This is an efficient way of handling potentially large collections of data while avoiding duplicates and unnecessary memory usage.

In [11]:
# Get Llama 3.2 to answer
ollama_via_openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

response = ollama_via_openai.chat.completions.create(
    model=MODEL_LLAMA,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question},
    ],
)

result = response.choices[0].message.content
display(Markdown(result))

**Breaking Down the Code**
==========================

This line of code uses a combination of Python features to extract author names from a list of dictionaries, `books`. It's a powerful snippet that can be broken down into smaller components for better understanding.

### Code Components

Let's break down the code into its individual components:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

*   `{...}`: This is a dictionary comprehension (also known as an expression comprehension) that generates a new list of authors. The syntax was introduced in Python 3.5.
*   `for book in books`: This is a loop over the `books` list, iterating over each dictionary (`book`) in the collection.
*   `if book.get("author")`: This condition filters the results to only include dictionaries that have an `"author"` key.
*   `yield from {...}`: This keyword uses the generator expression syntax with the `yield from` statement.

### What's Happening Here?

Here's a step-by-step explanation of what happens when this code is executed:

1.  **Filtering the Data**: The initial condition, `if book.get("author")`, ensures that only dictionaries containing an `"author"` key are considered for inclusion in the resulting list.
2.  **Generating Author Names as a Dictionary Comprehension**: Inside the loop and using the dictionary comprehension `{...}`, each filtered `book` is used to generate its author name (`book.get("author")`). This creates a new dictionary that contains only the author names.

However, this might lead to an error if there are values in your dataset other than strings (like integers or lists):
```python
books = [
    {"id": 1, "name": "Book One", "author_id": 111},
    {"id":2 ,"name":" Book Two"," author:"Author Three"}
    
]
yield from {book.get("author") for book in books if book.get("author")}
# returns {"Author Three":["Book One"," Book Two"]}
```

3.  **Yielding a Sequence of Author Names**: The `yield from` statement is used to yield a generator expression, which produces the author names one by one.

In summary, this line of code effectively extracts the author names (if present in each dictionary) and returns them as a sequence from another sequence using the `yield from` syntax with a dictionary comprehension.

### Python Best Practices

Here are some best practices:

*   Make sure to consider edge cases (e.g., empty dictionaries or unexpected key values).

**Example Code**
```python
# Example dataset of book objects with author information
books = [
    {"id": 1, "name": "Book One", "author_id": 111},
    {"id": 2, "name":" Book Two"," author:"Author Three"},
    # more books here...
]

def get_authors():
    """Filtered list of authors"""
    return yield from {book.get("author") for book in books if book.get("author")}

# Execute and iterate over the generator (e.g., on a file or in-line function usage)
for author_name in get_authors():
    print(author_name)  # prints only those with actual data
```